## Поиск усердных учеников.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной". Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [1]:
#подключаемся к clickhouse
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

In [2]:
# создаем запрос, db автоматически заменится на значение из database
q = ''' 
    SELECT 
        count(st_id) as count_succed_id 
    FROM
        (
        SELECT 
            st_id,
            sum(correct) as sum_correct
        FROM 
            default.peas
        GROUP BY st_id
        HAVING sum_correct >= 20
        )
    '''
try:
    # отправляем запрос и записываем результат в пандасовский датафрейм
    q_test = ph.read_clickhouse(query=q, connection=connection_default)
    print(q_test)
except Exception:
    print("Не работает подключение к ClickHouse")

Не работает подключение к ClickHouse


## Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
- ARPU, 
- ARPAU, 
- CR в покупку, 
- СR активного пользователя в покупку, 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике, 

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [3]:
metrics = ''' 
    SELECT
    groups,
    round(sum(revenue)/uniq(st_id), 2) as ARPU,
    round(sum(revenue)/sum(active_users), 2) as ARPAU,
    round(sum(clients)/uniq(st_id)*100, 2) as CR,
    round(sum(active_users)/uniq(st_id)*100, 2) as CR_active,
    round(sum(math_clients)/sum(active_in_math)*100, 2) as CR_math
    
FROM(
    SELECT c.st_id AS st_id, b.revenue AS revenue, a.active AS active_users, 
    a.active_math AS active_in_math, b.clients AS clients, 
    b.math_clients AS math_clients, c.test_grp AS groups 
    
    FROM
        (SELECT a.st_id, a.active, b.active_math
        FROM
            (SELECT st_id,
            CASE WHEN sum_correct > 10 THEN 1 ELSE 0 END AS active 
            FROM
                (
                SELECT st_id, sum(correct) as sum_correct
                FROM default.peas
                GROUP BY st_id
                )
            ) as a
        LEFT JOIN  
            (SELECT st_id,
                CASE WHEN sum_correct >=2 THEN 1 ELSE 0 END AS active_math 
            FROM
                (
                SELECT st_id, sum(correct) as sum_correct
                FROM default.peas
                WHERE subject = 'Math'
                GROUP BY st_id
                )
            ) as b
            ON a.st_id = b.st_id
        ) as a
    
    LEFT JOIN
        (SELECT l.st_id, l.revenue, l.clients, r.math_clients
        FROM
            (
            SELECT st_id, sum(money) as revenue, 1 as clients 
            FROM final_project_check
            GROUP BY st_id 
            ) as l
        LEFT JOIN
            (
            SELECT st_id, 1 as math_clients
            FROM final_project_check
            WHERE subject = 'Math'
            GROUP BY st_id 
            ) as r
        on l.st_id = r.st_id
        )as b
        ON a.st_id=b.st_id
    
    RIGHT JOIN studs as c
    ON a.st_id=c.st_id
    )
GROUP BY groups 
    '''
# отправляем запрос и записываем результат в пандасовский датафрейм
try:
    metrics_test = ph.read_clickhouse(query=metrics, connection=connection_default)
    print(metrics_test)
except Exception:
    print("Connection Error")

Connection Error
